# Poop Power: Beta Diversity Analysis, Rarefaction and Significance Tests

In [3]:
# importing all required packages & notebook extensions at the start of the notebook
import os
import pandas as pd
import qiime2 as q2
import seaborn as sns
import matplotlib.pyplot as plt
from skbio import OrdinationResults
from qiime2 import Visualization as vis
from seaborn import scatterplot

%matplotlib inline

In [4]:
data_dir = 'poop_data/BetaDiversity'
div_dir = 'poop_data/Diversity'
phy_dir = 'poop_data/Phylogeny'
tox_dir = 'poop_data/Taxonomy'
base_dir = 'poop_data'
extract_dir = 'poop_data/BetaDiversity/uw_unifrac_significance_permanova'

## 1. Metadata



In [5]:
#get metadata as a dataframe
df_metadata = pd.read_csv('poop_data/metadata.tsv', sep = '\t')
#set sampleid as index
df_metadata.set_index('sampleid', inplace = True)
metadata_col = list(df_metadata.columns)
#excluding NaN-values
#df_metadata.isna().sum()
df_metadata = df_metadata.dropna()
len(df_metadata)

459

In [4]:
#len(original metadata) minus len(metadata without lines with missing values)
523-459

64

In [5]:
df_metadata.nunique()

GEN_age_cat                       8
GEN_age_corrected                71
GEN_bmi_cat                       4
GEN_bmi_corrected               343
GEN_cat                           2
GEN_collection_timestamp        446
GEN_country                      10
GEN_dog                           2
GEN_elevation                   335
GEN_geo_loc_name                 52
GEN_height_cm                    53
GEN_host_common_name              1
GEN_last_move                     6
GEN_last_travel                   6
GEN_latitude                    147
GEN_level_of_education            8
GEN_longitude                   167
GEN_race                          6
GEN_sample_type                   1
GEN_sex                           4
GEN_weight_kg                    80
HEA_acid_reflux                   2
HEA_add_adhd                      2
HEA_allergic_to_peanuts           2
HEA_antibiotic_history            6
HEA_appendix_removed              2
HEA_autoimmune                    2
HEA_bowel_movement_frequency

In [6]:
#list of columns with a) more than one value & b) categorical (you can only do the PERMANOVA with those ones) & c) the ones 
#which give an error during permanova
metadata_col_cat = ['GEN_age_cat', 'GEN_bmi_cat', 'GEN_cat',
 'GEN_dog', 'GEN_last_move', 'GEN_last_travel', 'GEN_level_of_education', 'GEN_race', 'GEN_sex',  
 'HEA_acid_reflux', 'HEA_add_adhd', 'HEA_allergic_to_peanuts', 'HEA_antibiotic_history', 'HEA_appendix_removed', 
 'HEA_autoimmune', 'HEA_bowel_movement_frequency', 'HEA_bowel_movement_quality', 'HEA_cancer', 'HEA_cancer_treatment', 
 'HEA_cardiovascular_disease', 'HEA_cdiff', 'HEA_chickenpox', 'HEA_csection', 'HEA_diabetes', 
 'HEA_exercise_frequency', 'HEA_ibd', 'HEA_ibs', 'HEA_liver_disease', 'HEA_lung_disease', 'HEA_mental_illness', 
 'HEA_migraine', 'HEA_seasonal_allergies', 'HEA_sibo', 'HEA_skin_condition', 'HEA_sleep_duration', 
'HEA_smoking_frequency', 'HEA_thyroid', 'HEA_weight_change']

metadata_col_intestine_disease = ['HEA_cdiff', 'HEA_ibd', 'HEA_ibs', 'HEA_sibo', 'HEA_acid_reflux']
metadata_col_intestine_detail = ['HEA_appendix_removed','HEA_bowel_movement_frequency', 
                                 'HEA_bowel_movement_quality']
metadata_col_MO_interruption = ['HEA_csection', 'HEA_antibiotic_history', 'GEN_last_move', 'GEN_last_travel', 
                                'HEA_weight_change']
metadata_col_lifestyle = ['GEN_cat','GEN_dog', 'HEA_exercise_frequency','HEA_sleep_duration', 'HEA_smoking_frequency']
metadata_col_disease_history = ['HEA_cancer_treatment','HEA_chickenpox']
metadata_col_active_disease = ['HEA_cancer','HEA_diabetes','HEA_thyroid','HEA_migraine', 'HEA_lung_disease', 'HEA_liver_disease', 'HEA_cardiovascular_disease']
metadata_col_allergy = ['HEA_autoimmune', 'HEA_seasonal_allergies','HEA_allergic_to_peanuts']
metadata_col_mental = ['HEA_add_adhd','HEA_mental_illness']
metadata_col_body = ['HEA_weight_change','GEN_age_cat', 'GEN_bmi_cat','GEN_race', 'GEN_sex']

## 2. Beta Diversity
### Principal Coordinates Plots PCoA
#### a) unweighted_unifrac_emperor

In [46]:
vis.load(f'{div_dir}/core-metrics-results/unweighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: a933dc03-1407-43fd-84b9-9cb91d0d24e6>

#### b) weighted_unifrac_emperor

In [25]:
vis.load(f'{div_dir}/core-metrics-results/weighted_unifrac_emperor.qzv')

<visualization: Visualization uuid: ddb6b230-5297-4263-80eb-d02a18a39f2b>

#### c) bray_curtis_emperor

In [45]:
vis.load(f'{div_dir}/core-metrics-results/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: 39adce00-c1d5-43d5-ae3e-b76cf35404c0>

#### d) jaccard emperor

In [26]:
vis.load(f'{div_dir}/core-metrics-results/jaccard_emperor.qzv')

<visualization: Visualization uuid: 4dbee86d-1ab5-4466-934b-449b2ad1ced0>

## 2. PERMANOVA
#### Anova Test 1: Are coordinates significantly different from each other with differing column values? looking at one column

could be further done: doing permanova also for weighted unifrac, jaccard and bray curtis to look at the differences

In [7]:
! qiime diversity beta-group-significance \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --m-metadata-column  GEN_bmi_cat \
    --p-pairwise \
    --o-visualization $data_dir/uw_unifrac-GEN_bmi_cat-significance.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-GEN_bmi_cat-significance.qzv


In [19]:
#doing the permanova test for all of the columns
for column in metadata_col_cat:
    ! qiime diversity beta-group-significance \
        --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
        --m-metadata-file $base_dir/metadata.tsv \
        --m-metadata-column  $column \
        --p-pairwise \
        --o-visualization $data_dir/uw_unifrac-$column-significance.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-GEN_age_cat-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-GEN_bmi_cat-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-GEN_cat-significance.qzv
Plugin error from diversity:

  All values in the grouping vector are unique. This method cannot operate on a grouping vector with only unique values (e.g., there are no 'within' distances because each group of objects contains only a single object).

Debug info has been saved to /tmp/qiime2-q2cli-err-n5i8dgrp.log
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-GEN_dog-significance.qzv
Plugin error from diversity:

  All values in the grouping vector are unique. This method cannot operate on a grouping vector with only unique values (e.g., there are no 'within' distances because each group of objects contains only a single object).

Debug info has been saved to /tmp/qiime2-q2cli-err-rad_o_d5.log
Saved Visualization to: 

In [44]:
vis.load(f'{extract_dir}/uw_unifrac-GEN_bmi_cat-significance.qzv')

<visualization: Visualization uuid: 7021e6be-11b9-4168-8206-afc4e67bd403>

In [9]:
###This does not work. how could it work ? 
for column in metadata_col_cat:
    Visualization.load(f'{data_dir}/uw_unifrac-{column}-significance.qzv')
    

In [42]:
vis.load(f'{extract_dir}/uw_unifrac-HEA_acid_reflux-significance.qzv')

<visualization: Visualization uuid: 4e021e15-0371-46bc-8941-b3854e84a2b9>

#### Anova Test 2: 
could be further done: doing permanova also for weighted unifrac, jaccard and bray curtis to look at the differences

In [21]:
#create an input for adonis-anova-test; listing all column titles of metadata

columns = ""
for column in metadata_col_cat:
    if column == "HEA_weight_change":
        columns = columns + column
    else:
        columns = columns + column + "+"
print(columns)

GEN_age_cat+GEN_bmi_cat+GEN_cat+GEN_dog+GEN_last_move+GEN_last_travel+GEN_level_of_education+GEN_race+GEN_sex+HEA_acid_reflux+HEA_add_adhd+HEA_allergic_to_peanuts+HEA_antibiotic_history+HEA_appendix_removed+HEA_autoimmune+HEA_bowel_movement_frequency+HEA_bowel_movement_quality+HEA_cancer+HEA_cancer_treatment+HEA_cardiovascular_disease+HEA_cdiff+HEA_chickenpox+HEA_csection+HEA_diabetes+HEA_exercise_frequency+HEA_ibd+HEA_ibs+HEA_liver_disease+HEA_lung_disease+HEA_mental_illness+HEA_migraine+HEA_seasonal_allergies+HEA_sibo+HEA_skin_condition+HEA_sleep_duration+HEA_smoking_frequency+HEA_thyroid+HEA_weight_change


In [76]:
df_metadata.to_csv(f'{data_dir}/metadata_dropna.csv', sep = '\t')  
df_metadata3 = pd.read_csv(f'{data_dir}/metadata_dropna.csv', sep = '\t')

,sampleid,GEN_age_cat,GEN_age_corrected,GEN_bmi_cat,GEN_bmi_corrected,GEN_cat,GEN_collection_timestamp,GEN_country,GEN_dog,GEN_elevation,...,HEA_lung_disease,HEA_mental_illness,HEA_migraine,HEA_seasonal_allergies,HEA_sibo,HEA_skin_condition,HEA_sleep_duration,HEA_smoking_frequency,HEA_thyroid,HEA_weight_change
0,10317.000046,20s,20.0,Normal,23.75,False,2016-08-25 18:30:00,USA,True,1919.3,...,False,False,False,True,False,False,8 or more hours,Never,False,Decreased more than 10 pounds
1,10317.000038,30s,39.0,Overweight,27.67,False,2016-06-29 09:30:00,United Kingdom,False,44.5,...,False,False,False,False,False,False,7-8 hours,Not provided,False,Remained stable
2,10317.000047,50s,56.0,Normal,19.71,False,2016-07-12 17:30:00,Germany,False,8.7,...,False,False,True,False,False,True,6-7 hours,Never,True,Decreased more than 10 pounds
3,10317.000046,40s,45.0,Normal,23.15,False,2016-05-24 19:00:00,United Kingdom,True,68.8,...,False,False,False,True,False,True,6-7 hours,Never,False,Remained stable
4,10317.000046,40s,46.0,Overweight,27.46,False,2016-07-07 08:10:00,United Kingdom,True,119.6,...,False,False,False,False,False,False,6-7 hours,Never,True,Not provided


In [79]:
missing_samples = ['10317.000047381', '10317.000036431', '10317.000053480', '10317.000054310', 
                   '10317.000047230', '10317.000054330', '10317.000039980', '10317.000030366', 
                   '10317.000047370', '10317.000031598', '10317.000042660', '10317.000062070', 
                   '10317.000044340', '10317.000050288', '10317.000047228', '10317.000040490', 
                   '10317.000046290', '10317.000046305', '10317.000047404', '10317.000051558', 
                   '10317.000036170', '10317.000037960', '10317.000002930', '10317.000053353', 
                   '10317.000053458', '10317.000047151', '10317.000051160', '10317.000048326', 
                   '10317.000048283', '10317.000047140', '10317.000053435', '10317.000033294', 
                   '10317.000042590', '10317.000052055', '10317.000041592', '10317.000042635', 
                   '10317.000051560', '10317.000042655', '10317.000053430', '10317.000050273', 
                   '10317.000047606', '10317.000050240', '10317.000030383', '10317.000027920', 
                   '10317.000047141', '10317.000028654', '10317.000052260', '10317.000046121',
                   '10317.000050156', '10317.000052034', '10317.000054323', '10317.000051258', 
                   '10317.000044550', '10317.000042969', '10317.000062073', '10317.000038019',
                   '10317.000058480', '10317.000046336', '10317.000052280', '10317.000052450', 
                   '10317.000051561', '10317.000051130', '10317.000041730', '10317.000050294', 
                   '10317.000052030', '10317.000052380', '10317.000047777', '10317.000048284',
                   '10317.000047680', '10317.000030384', '10317.000047610', '10317.000047380', 
                   '10317.000051210', '10317.000047196', '10317.000050290', '10317.000051100', 
                   '10317.000047220', '10317.000058550', '10317.000044252', '10317.000053433', 
                   '10317.000053410', '10317.000054313', '10317.000042865', '10317.000062083', 
                   '10317.000062076', '10317.000038081', '10317.000051180', '10317.000052448', 
                   '10317.000047197', '10317.000042845', '10317.000052431', '10317.000030255',
                   '10317.000048277', '10317.000032650', '10317.000047620', '10317.000052370', 
                   '10317.000052432', '10317.000053310', '10317.000046270']
len(missing_samples)

99

**This is weird though, as the .dropna() command only deleted 63 lines and not 99. where did some of the other lines got lost?**

In [22]:
#problem here: it was not happy that there is missing data in the df. I deleted the lines with missing data. 
#now it is complaining that there are lines missing I guess (and i just had to delete them as they were empty. 
#how to solve that? replacing NaN values with fake values? this is not really possible for boolean columns I 
#guess...)
! qiime diversity adonis \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --p-formula $columns \
    --o-visualization $data_dir/uw_unifrac-overall-significance-adonis.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-overall-significance-adonis.qzv


In [24]:
vis.load(f'{data_dir}/uw_unifrac-overall-significance-adonis.qzv')

<visualization: Visualization uuid: 6a915077-6ffa-4850-8bf1-92bc3486b53a>

In [8]:
! qiime diversity adonis \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --p-formula  'HEA_cdiff+HEA_ibd+HEA_ibs+HEA_sibo+HEA_acid_reflux'\
    --o-visualization $data_dir/uw_unifrac-cdiff_ibd_ibs_sibo_acidrefl-significance-adonis.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-cdiff_ibd_ibs_sibo_acidrefl-significance-adonis.qzv


In [9]:
vis.load(f'{data_dir}/uw_unifrac-cdiff_ibd_ibs_sibo_acidrefl-significance-adonis.qzv')

<visualization: Visualization uuid: 62d7644e-1065-4906-81c0-7391b1c4979e>

In [11]:
! qiime diversity adonis \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --p-formula  'HEA_csection+HEA_antibiotic_history+GEN_last_move+GEN_last_travel+HEA_weight_change' \
    --o-visualization $data_dir/uw_unifrac-MO_interruption-significance-adonis.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-MO_interruption-significance-adonis.qzv


In [12]:
vis.load(f'{data_dir}/uw_unifrac-MO_interruption-significance-adonis.qzv')

<visualization: Visualization uuid: fc932880-bc85-4272-9a89-5cf43e33eb58>

In [13]:
! qiime diversity adonis \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --p-formula  'HEA_cancer+HEA_diabetes+HEA_thyroid+HEA_migraine+HEA_lung_disease+HEA_liver_disease+HEA_cardiovascular_disease' \
    --o-visualization $data_dir/uw_unifrac-active_diseases-significance-adonis.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-active_diseases-significance-adonis.qzv


In [14]:
vis.load(f'{data_dir}/uw_unifrac-active_diseases-significance-adonis.qzv')

<visualization: Visualization uuid: b6d9dabd-0c58-4973-b4fb-4e8dbfe4a818>

In [15]:
! qiime diversity adonis \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --p-formula  'GEN_cat+GEN_dog+HEA_exercise_frequency+HEA_sleep_duration+HEA_smoking_frequency' \
    --o-visualization $data_dir/uw_unifrac-lifestyle-significance-adonis.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-lifestyle-significance-adonis.qzv


In [16]:
vis.load(f'{data_dir}/uw_unifrac-lifestyle-significance-adonis.qzv')

<visualization: Visualization uuid: 54053267-203e-47c3-acb4-9e0e66b6f58f>

In [17]:
! qiime diversity adonis \
    --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file $base_dir/metadata.tsv \
    --p-formula  'HEA_appendix_removed+HEA_bowel_movement_frequency+HEA_bowel_movement_quality' \
    --o-visualization $data_dir/uw_unifrac-intestine_detail-significance-adonis.qzv

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac-intestine_detail-significance-adonis.qzv


In [18]:
vis.load(f'{data_dir}/uw_unifrac-intestine_detail-significance-adonis.qzv')

<visualization: Visualization uuid: d0b6b088-1530-4dd5-a840-f9399d956c18>

adonis: einfach nochmals probieren mit weniger spalten auf einmal (vlt schlau gruppieren),
    vlt die GEN fakroten mit multiplikation dazu nehmen wegen interaktionen? ausprobieren...
    und sonst wenn man nicht  davon ausgeht dass es interaktionen gibt dann einfach mit plus

In [1]:
! qiime diversity adonis --help

Usage: qiime diversity adonis [OPTIONS]

  Determine whether groups of samples are significantly different from one
  another using the ADONIS permutation-based statistical test in vegan-R.
  The function partitions sums of squares of a multivariate data set, and is
  directly analogous to MANOVA (multivariate analysis of variance). This
  action differs from beta_group_significance in that it accepts R formulae
  to perform multi-way ADONIS tests; beta_group_signficance only performs
  one-way tests. For more details, consult the reference manual available on
  the CRAN vegan page: https://CRAN.R-project.org/package=vegan

Inputs:
  --i-distance-matrix ARTIFACT
    DistanceMatrix     Matrix of distances between pairs of samples.
                                                                    [required]
Parameters:
  --m-metadata-file METADATA...
    (multiple          Sample metadata containing formula terms.
     arguments will    
     be merged)                                 

In [20]:
metadata_col_cat_trial = ['HEA_cardiovascular_disease', 'HEA_cdiff']

In [29]:
#empty dataframe for adding the other dataframes with q-value-informations to it
df_empty = pd.DataFrame(columns = ['Group 1', 'Group 2', 'Sample size', 'Permutations', 'pseudo-F', 
                                   'p-value', 'q-value', 'column'])

for column in metadata_col_cat:
    #producing path name:
    path = f'{extract_dir}/uw_unifrac-{column}-significance.qzv'
    name = f'uw_unifrac-{column}-significance.qzv'
    
    #permanova visualisation production for every column
    ! qiime diversity beta-group-significance \
        --i-distance-matrix $div_dir/core-metrics-results/unweighted_unifrac_distance_matrix.qza \
        --m-metadata-file $base_dir/metadata.tsv \
        --m-metadata-column  $column \
        --p-pairwise \
        --o-visualization $data_dir/uw_unifrac_significance_permanova/uw_unifrac-$column-significance.qzv
    
    #extract the qzv that was just produced
    vis.extract(f'{extract_dir}/{name}', output_dir = f'{extract_dir}/permanova_extracted')
    
    #get the uuid of it
    uid = vis.peek(f'{extract_dir}/{name}').uuid
    
    #go to the folder of the uuid and turn the permanova_paired table as a dataframe
    df_permanova = pd.read_csv(f'{extract_dir}/permanova_extracted/{uid}/data/permanova-pairwise.csv')
    
    #add a new column for the assignment to the original column
    df_permanova['column'] = column
    
    #concat the new dataframe with the ones before
    df_empty = pd.concat([df_empty, df_permanova])
    
#close the loop
    
df_empty

Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_age_cat-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_bmi_cat-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_cat-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_dog-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_last_move-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_last_travel-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_level_of_education-significance.qzv
Saved Visualization to: poop_data/BetaDiversity/uw_unifrac_significance_permanova/uw_unifrac-GEN_race-significance.qzv
Saved Visualizatio

,Group 1,Group 2,Sample size,Permutations,pseudo-F,p-value,q-value,column
0,20s,30s,119,999,1.429988,0.027,0.074769,GEN_age_cat
1,20s,40s,149,999,2.213569,0.002,0.012000,GEN_age_cat
2,20s,50s,164,999,3.120116,0.001,0.012000,GEN_age_cat
3,20s,60s,124,999,2.849373,0.002,0.012000,GEN_age_cat
4,20s,70+,78,999,2.567982,0.001,0.012000,GEN_age_cat
...,...,...,...,...,...,...,...,...
1,Decreased more than 10 pounds,Not provided,64,999,1.247124,0.110,0.165000,HEA_weight_change
2,Decreased more than 10 pounds,Remained stable,461,999,1.434701,0.027,0.054000,HEA_weight_change
3,Increased more than 10 pounds,Not provided,48,999,1.133423,0.215,0.258000,HEA_weight_change
4,Increased more than 10 pounds,Remained stable,445,999,1.509623,0.026,0.054000,HEA_weight_change


In [35]:
df_empty.to_csv(f'{data_dir}/uw_unifrac_significance.csv')

In [40]:
#these are all the paired comparisons from permanova, where the q-value is below significance-level
df_empty[df_empty['q-value']<0.05]

,Group 1,Group 2,Sample size,Permutations,pseudo-F,p-value,q-value,column
1,20s,40s,149,999,2.213569,0.002,0.012000,GEN_age_cat
2,20s,50s,164,999,3.120116,0.001,0.012000,GEN_age_cat
3,20s,60s,124,999,2.849373,0.002,0.012000,GEN_age_cat
4,20s,70+,78,999,2.567982,0.001,0.012000,GEN_age_cat
9,30s,50s,195,999,1.702828,0.006,0.024000,GEN_age_cat
10,30s,60s,155,999,1.882947,0.004,0.018000,GEN_age_cat
11,30s,70+,109,999,1.888061,0.002,0.012000,GEN_age_cat
20,40s,teen,116,999,1.644526,0.014,0.045818,GEN_age_cat
25,50s,teen,131,999,2.020266,0.002,0.012000,GEN_age_cat
29,60s,teen,91,999,2.045634,0.004,0.018000,GEN_age_cat
